In [15]:
# pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [16]:
import os 
os.environ["LANGCHAIN_API_KEY"] = "..."
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['OPENAI_API_KEY'] = '...'



In [17]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_ = ('post-content','post-title','post-header')
        )
    ),   
)


blog_docs = loader.load()

## Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap=50
)

blog_splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=blog_splits,
    embedding = OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

# Part 5: Query_Translation

## 1) Multi Query

In [18]:
from langchain.prompts import ChatPromptTemplate

template = '''You are an AI language model assistant. 
Your task is to generate five different 
versions of the given user question to retrieve 
relevant documents from a vector database. 
By generating multiple perspectives on the user question, 
your goal is to help the user overcome some of the limitations 
of the distance-based similarity search. Provide these 
alternative questions separated by newlines. 
Original question: {question}'''

prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives
    |ChatOpenAI(temperature = 0.9)
    |StrOutputParser()
    |(lambda x:x.split('\n')) 
)
# returns list of 5 questions

In [19]:
from langchain.load import dumps, loads

# dumps -> converts each doc to string
# loads -> converts each string back to doc
def get_unique_union(documents: list[list]):
    '''unique union of retrieved documents'''
    # flatten list of lists, and convert each Document to json string for comparision
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]


    #Get unique documents
    unique_docs = list(set(flattened_docs))

    return [loads(doc) for doc in unique_docs]

question = "what is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question": question})
len(docs)

# Retriever brings multiple documents for eact question and stores them in list, 
# therefore list[list]

#The map(list) applies a the retiever to each 
# element in a list (in this case, the list of 
# alternative questions) and returns a new collection with the results.

3

In [20]:
print(docs)

[Document(page_content='Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}), Document(page_content='The system comprises of 4 stages:\n(1) Task planning: LLM works as the brain and parses the user requests into multiple tasks. There are four attributes associated with each task: task type, ID, dependencies, and arguments. They use few-shot examples to guide LLM to do task parsing and planning.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}), Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'})]


In [21]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = '''Answer the following questions based on this context:
{context}

Question : {Ques}
'''

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature = 0)

final_rag_chain = (
    {"context":retrieval_chain, "Ques":itemgetter('question')} ## itemgetter('question) can be replaced by RunnablePassthrough()
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({'question':question})

'Task decomposition for LLM agents involves breaking down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.'

## 2) RAG-Fusion

Prompt

In [22]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = '''Your are a helpful assitant that generates multiple search queries based 
on a single query. \n
Generate multiple Serach queries related to: {question}\n
Output(4 querires):'''

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries=(
    prompt_rag_fusion
    |ChatOpenAI(temperature=0)
    |StrOutputParser()
    |(lambda x: x.split("\n"))
)

RRF works by taking the search results from multiple methods, assigning a reciprocal rank score to each document in the results, and then combining the scores to create a new ranking. The concept is that documents appearing in the top positions across multiple search methods are likely to be more relevant and should be ranked higher in the combined result.

smaller the rank, greater the relevance. Therefore, inversely proportonal

In [24]:
from langchain.load import dumps,loads
def reciprocal_rank_fusion(results: list[list],k=60):
    '''takes multiple lists of ranked docs
    and an optional parameter k used in the RRF formula'''

    # Initailize a dict to hold fused scores for each unique documunet
    fused_scores = {}

    for docs in results:
        # Iterate through each document in the list, with its rank(position in the list)
        for rank,doc in enumerate(docs):
            # Convert the document to a string format to use as a key(assumes documnets can be serialized to JSON) 
            doc_str = dumps(doc)
            # If the dcoument is not yet in the fused_scores dictionary, add it withan initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retreive the current score of the document if any
            previous_score = fused_scores[doc_str]
            # Update the score pf the document using the RRF foumula: 1/(rank+k)
            fused_scores[doc_str] += 1/(rank+k)

        # Sort the documents based on their fused scores in descending order to get the final reranked results
        reranked_results = [
            (loads(doc),score)
            for doc,score in sorted(fused_scores.items(), key=lambda x:[1], reverse = True)
        ]

        return reranked_results
    
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question":question})


In [25]:
docs

[(Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
  0.03306010928961749),
 (Document(page_content='(2) Model selection: LLM distributes the tasks to expert models, where the request is framed as a multiple-choice question. LLM is presented with a list of models to choose from. Due to the limited context length, task type based filtration is needed.\nInstruction:', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
  0.03200204813108039)]

In [26]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template ='''Answer the followig question based on the context:
{context}
Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion,
     'question': itemgetter('question')}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({'question':question})


'Task decomposition for LLM agents involves breaking down a complicated task into smaller steps that the agent can understand and plan ahead for. This allows the agent to effectively navigate through the task and make informed decisions.'

## 3) Query Translation (Decomposition)

### Type 1. answers of one question are relevant to the next question

In [30]:
from langchain.prompts import ChatPromptTemplate

#Decomposition
template = '''You are a helpful assistant that generates multiple sub-queries related toa n input question.\n
The Goal is to break down the input into a set of sub-problems/ sub-queries that can be answerd in isolation. \n
Generate multiple search queries related to: {question}\n
Output(3 queries):'''

prompt_decomposition = ChatPromptTemplate.from_template(template)

In [32]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# LLm
llm = ChatOpenAI(temperature=0)

# Chain
generate_queries_decomposition = (prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split('\n') ) )

# RUN
question = 'what are the main components of an LLM-powered autonomous agent system?'

questions = generate_queries_decomposition.invoke({"question":question})

In [33]:
questions

['1. What is LLM technology and how does it work in autonomous agent systems?',
 '2. What are the key components of an autonomous agent system powered by LLM technology?',
 '3. How do LLM-powered autonomous agent systems differ from other types of autonomous systems in terms of their components and functionality?']

In [34]:
# Prompt 

template = '''Here is the question you need to answer:

\n --- \n {question}\n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs}

here is additional context relevant to the question:

\n --- \n {context}\n --- \n

Use the above context and any background + answer pairs to answer the question: \n {question}
'''

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [35]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question,answer):
    '''Format Q and A pair'''

    formatted_string = ''
    formatted_string += f"Question: {question}\n Answer: {answer}\n\n"
    return formatted_string.strip()

# llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0)

#
q_a_pairs = ''
for q in questions:
    rag_chain = (
        {'context':itemgetter('question')|retriever,
         'question': itemgetter('question'),
         'q_a_pairs': itemgetter('q_a_pairs')}
         |decomposition_prompt
         |llm
         |StrOutputParser()
    )

    answer = rag_chain.invoke({'question':q,'q_a_pairs':q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + '\n------\n'+ q_a_pair

In [37]:
print(answer) #last answer in the list 

LLM-powered autonomous agent systems differ from other types of autonomous systems in terms of their components and functionality by having LLM technology as a central component. LLM functions as the agent's brain, enabling it to process and understand human language inputs, make informed decisions, and carry out tasks effectively. This advanced AI model allows the agent to interpret and respond to natural language inputs, which is a unique feature not commonly found in other autonomous systems. Additionally, LLM-powered autonomous agent systems often include planning mechanisms to help the agent decompose complex tasks, plan ahead, and execute tasks in a structured manner. These components work together to enhance the agent's cognitive abilities, decision-making processes, and task execution capabilities, setting them apart from other autonomous systems that may not have such advanced language processing and planning capabilities.


### Type 2. Sub-answers are not related.

In [51]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    '''RAG on each sub-question'''

    # Use decompositiom
    sub_questions = sub_question_generator_chain.invoke({"question":question})

    # Initialize a list to hold RAG chain results

    rag_results = []

    for sub_question in sub_questions:

        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)

        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({'context':retrieved_docs,
                                                                'question':sub_question})
        rag_results.append(answer)

    return rag_results, sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers , questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

['1. What is LLM technology and how does it work in autonomous agent systems?', '2. What are the key components of an autonomous agent system powered by LLM technology?', '3. How do the main components of an LLM-powered autonomous agent system interact with each other to enable autonomous behavior?']
1. What is LLM technology and how does it work in autonomous agent systems?
2. What are the key components of an autonomous agent system powered by LLM technology?
3. How do the main components of an LLM-powered autonomous agent system interact with each other to enable autonomous behavior?


In [52]:
len(answers)

3

In [60]:
def format_qa_pairs(questions,answers):

    formatted_string = ''
    for (question,answer) in zip(questions,answers):
        formatted_string += f"Question: {question}\n Answer: {answer}\n\n"
        print(question,'\n',answer)
        print(formatted_string)
    return formatted_string.strip()
    
context = format_qa_pairs(questions, answers)

# Prompt

template = '''Here is the set of Q+A pairs:
{context}

Use these to synthesize an answer to the question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
        prompt|
        llm|
        StrOutputParser()
    )
    
final_rag_chain.invoke({"context":context ,'question':question})

1. What is LLM technology and how does it work in autonomous agent systems? 
 LLM technology functions as the brain of autonomous agent systems, supported by key components. In these systems, LLM helps the agent plan and execute tasks by decomposing complex tasks into manageable steps. It serves as a central component in guiding the agent's decision-making and actions.
Question: 1. What is LLM technology and how does it work in autonomous agent systems?
 Answer: LLM technology functions as the brain of autonomous agent systems, supported by key components. In these systems, LLM helps the agent plan and execute tasks by decomposing complex tasks into manageable steps. It serves as a central component in guiding the agent's decision-making and actions.


2. What are the key components of an autonomous agent system powered by LLM technology? 
 The key components of an autonomous agent system powered by LLM technology include LLM as the agent's brain and other components such as planning a

"The main components of an LLM-powered autonomous agent system include the LLM technology serving as the agent's brain, planning for task understanding and preparation, and task decomposition for breaking down complex tasks into manageable steps. These components work together to enable autonomous behavior by guiding the agent's decision-making and actions in executing tasks effectively."

## 4) Query Translation (step-back)
output more generic questions which are easier to answer

In [61]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)
generate_queries_step_back = prompt | ChatOpenAI(temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})
# Response prompt 
response_prompt_template = """You are an expert of world knowledge. 
I am going to ask you a question. Your response should be comprehensive 
and not contradicted with the following context if they 
are relevant. Otherwise, ignore them if 
they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition for LLM agents refers to the process of breaking down large and complex tasks into smaller, more manageable subgoals. This approach enables LLM-powered autonomous agents to efficiently handle intricate tasks by dividing them into smaller components that can be tackled sequentially or in parallel. By decomposing tasks, LLM agents can better understand the steps involved in completing a task and plan ahead accordingly. This process helps the agents to navigate through various dependencies and requirements associated with the task, leading to more effective and successful task execution.\n\nTask decomposition is a crucial component of the overall functioning of LLM-powered autonomous agent systems. It allows the agents to parse user requests, identify the different tasks required to fulfill those requests, and organize them in a structured manner. By utilizing few-shot examples and leveraging the capabilities of LLM as the brain of the system, task decomposition ensure

## 5) Query Translation (HyDE)

-- the users questions are converted to a hypothetical document that is mapped to the real document.

-- Because the hypothetical document will be closer to the real document than the user question.

In [62]:
from langchain.prompts import ChatPromptTemplate

# HyDE 
template = '''please write a scientifc paper passge to answer the question
Question:{question}
Passage:'''
prompt_hyde = ChatPromptTemplate.from_template(template)

generate_docs_for_retrieval = (
    prompt_hyde
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

question = 'what is task decomposition for LLM agents?'
generate_docs_for_retrieval.invoke({"question": question})

'Task decomposition is a fundamental concept in the field of machine learning and artificial intelligence, particularly for agents utilizing the Long-Short Term Memory (LSTM) architecture. Task decomposition refers to the process of breaking down a complex task into smaller, more manageable sub-tasks that can be individually learned and executed by the agent. \n\nIn the context of LLM agents, task decomposition is crucial for improving the efficiency and effectiveness of the learning process. By breaking down a complex task into smaller sub-tasks, the agent can focus on learning and optimizing each sub-task separately, before combining them to perform the overall task. This approach allows the agent to learn more quickly and accurately, as it can leverage its memory capabilities to retain information about each sub-task and apply it to the larger task.\n\nFurthermore, task decomposition can also help LLM agents generalize better to new, unseen tasks. By learning a set of basic sub-task

In [63]:
retrieval_chain = generate_docs_for_retrieval | retriever
retrieved_docs = retrieval_chain.invoke({"question": question})

In [64]:
retrieved_docs

[Document(page_content='Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are a

In [65]:
# RAG

template = '''Answer the following question based on this context:

{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt|
    llm|
    StrOutputParser()
)

final_rag_chain.invoke({"context":retrieved_docs, 'question':question})

'Task decomposition for LLM agents involves breaking down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks. This process allows the agent to plan ahead and understand the steps involved in completing a complicated task.'